In [7]:
!pip install hyperopt

  Using cached hyperopt-0.2.7-py2.py3-none-any.whl (1.6 MB)
     |████████████████████████████████| 199 kB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 10.4 MB/s eta 0:00:01
  Using cached cloudpickle-2.1.0-py3-none-any.whl (25 kB)
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=ac71af753f3c1160c6a0b3f5007c35795d87af60b0b217ad8b4d6a86beacf5a8
  Stored in directory: /Users/przivic/Library/Caches/pip/wheels/2f/a0/d3/4030d9f80e6b3be787f19fc911b8e7aa462986a40ab1e4bb94
Successfully built future


In [2]:
import sys
sys.path.append('../../lib')

In [3]:
from taller_model_selection.serialize import iter_jl

In [4]:
X, y = map(list, map(iter_jl, ['X_train.jl', 'y_train.jl']))

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)
X_dev, X_test, y_dev, y_test = train_test_split(X_test, y_test, test_size=0.5)
len(X_train), len(y_train), len(X_dev), len(y_dev), len(X_test), len(y_test)

(121272, 121272, 20212, 20212, 20212, 20212)

In [6]:
from taller_model_selection.metrics import rmse

In [9]:
from taller_model_selection.transformers import FeatureProjection
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

features_pipe = make_union(
    make_pipeline(
        FeatureProjection(['rooms', 'bedrooms', 'bathrooms', 'surface_total', 'surface_covered']),
        SimpleImputer()
    ),
    make_pipeline(
        FeatureProjection(['l3']), 
        SimpleImputer(strategy='most_frequent'),
        OneHotEncoder(sparse=False, drop='first')
    ), 
    make_pipeline(
        FeatureProjection(['l4']), 
        SimpleImputer(strategy='constant'),
        OneHotEncoder(sparse=False, drop='first')
    ), 
)

In [23]:
from hyperopt import fmin, tpe, hp, STATUS_OK
from time import time

def eval_pipe(model_name, pipe):
    return dict(
        name=model_name,
        train=rmse(y_train, pipe.predict(X_train)),
        dev=rmse(y_dev, pipe.predict(X_dev))
    )

def rf_objective(params):
    rf_pipe = make_pipeline(
        features_pipe,
        RandomForestRegressor(n_estimators=20)
    )
    t0 = time()
    rf_pipe.fit(X_train, y_train)
    train_time = time() - t0

    return dict(
        loss=rmse(y_dev, rf_pipe.predict(X_dev)), 
        tr_loss=rmse(y_train, rf_pipe.predict(X_train)), 
        train_time=train_time,
        statis=STATUS_OK
    )

In [24]:

space = {
    'n_estimators': hp.uniform('n_estimators', 10, 30),
    'min_samples_split': hp.quniform('min_samples_split', 0, 0.3, 0.01),
    'criterion': hp.choice('criterion', ["squared_error", "absolute_error", "poisson"])
    
}
# minimize the objective over the space
from hyperopt import fmin, tpe
best = fmin(rf_objective, space, algo=tpe.suggest, max_evals=100)


  0%|                                   | 0/100 [00:00<?, ?trial/s, best loss=?]

job exception: 'status'



  0%|                                   | 0/100 [00:12<?, ?trial/s, best loss=?]


KeyError: 'status'